In [51]:
import re
import time
import multiprocessing
import threading
import logging
import os
from pytube import YouTube
from moviepy.editor import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from textblob import TextBlob
import spacy,nltk
from nrclex import NRCLex
from googletrans import Translator


In [52]:
# List to store the extracted video names
videoenames = []

##### The above obejct is intended to store the names of the YouTube videos that were downloaded. This could facilitate further analysys and visualization.

In [53]:
# Create a lock object to ensure only one thread writes to the file at a time.
file_lock = threading.Lock()

##### The above lock creates a mutex (mutual exclusion) lock for handling concurrency in multi-threaded processing. 

In [54]:
# Ensure the directories exist before configuring the logger:

# Directory to save videos
log_directory = 'video_records'
os.makedirs(log_directory, exist_ok=True)
# Directory to save downloaded videos
download_directory = 'downloaded_videos'
os.makedirs(download_directory, exist_ok=True)
# Directory to save extracted audio files
audio_directory = 'audios'
os.makedirs(audio_directory, exist_ok=True)
# Directory to save transcriptions
text_directory = 'texts'
os.makedirs(text_directory, exist_ok=True)

##### In the abnove section, all necessary directories for the current project are created. This is crucial for organizing the files, such as video recordings, downloaded videos, extracted audio files, and transcriptions. By using os.makedirs with the exist_ok=True parameter, these directories are created if they do not already exist. This prevents errors if the directories are missing and avoids recreating them if they are already present.

In [55]:
# Configure logging
logging.basicConfig(
    filename=os.path.join(log_directory, 'download_log.txt'),
    level=logging.INFO,
    format='"Timestamp": %(asctime)s, "URL":"%(url)s", "Thread": "%(threadName)s"',
    datefmt='%H:%M, %d %b %Y'
)
logger = logging.getLogger()

#### Setting Up Logging Configuration
The `logging.basicConfig` function sets up the basic configuration for the logging system.

#### Filename
`filename=os.path.join(log_directory, 'download_log.txt')`
- Specifies the file where the log entries will be stored.
- `log_directory` is a predefined directory where the log files are stored.
- The log file `download_log.txt` is created (or appended if it already exists) in the specified directory.

#### Logging Level
`level=logging.INFO`
- Sets the logging level to `INFO`.
- It means that all log messages at the level of INFO and above (WARNING, ERROR, CRITICAL) will be captured. Messages with a lower level (DEBUG) will not be logged.

#### Log Message Format
`format='"Timestamp": %(asctime)s, "URL":"%(url)s", "Thread": "%(threadName)s"'`
- Defines the format of the log messages.
- `%(asctime)s`: Inserts the timestamp of the log entry.
- `%(url)s`: A custom field intended to log the URL (this needs to be manually added to log messages).
- `%(threadName)s`: Inserts the name of the thread that generated the log message.

#### Date Format
`datefmt='%H:%M, %d %b %Y'`
- Specifies the format of the date and time in the log messages.
- For example, a log entry might have a timestamp like `14:32, 10 Jun 2024`.

#### Creating Logger Object
`logger = logging.getLogger()`
- Creates a logger object named `logger` which can be used throughout the script to log messages.

In [56]:
def video_record(url, thread_name):
    # Create path to the log file (download_log.txt) located within a directory specified by log_directory
    file_path = os.path.join(log_directory, 'download_log.txt')
    # Ensure threads don't write simultaneously
    with file_lock:
         # Open the file in append mode.
        with open(file_path, "a") as file:
            # Create a log entry with the generated phrase
            log_entry = f'"Timestamp": {time.strftime("%H:%M, %d %b %Y")}, "URL":"{url}", "Thread":"{thread_name}"\n'
             # Write the log entry to the file
            file.write(log_entry)

#### Function Definition
The function takes two arguments:
- `url`: The URL of the video being downloaded.
- `thread_name`: The name of the thread that is handling the download.

#### Creating the Path to the Log File
`file_path` constructs the full path to the log file `download_log.txt` within `log_directory`.

#### Ensuring Thread Safety
`file_lock` is a threading lock object that ensures that only one thread writes to the log file at a time. This prevents race conditions and ensures thread safety.

#### Writing to the Log File
The log file is opened in append mode (`"a"`), which means new log entries are added to the end of the file without overwriting existing content.
A log entry string is created, including the current timestamp, the URL, and the thread name. The timestamp format is `"%H:%M, %d %b %Y"`.
The log entry is then written to the file.

In [57]:
# Combines the downloading and logging steps into a single function.
def download_video_and_log(url):
    download_video(url)
    # Record which video was downloaded by which process or thread
    thread_name = threading.current_thread().name
    video_record(url, thread_name)

#### Function Definition
The function takes a single argument, `url`, which is the URL of the video to be downloaded.

#### Downloading the Video
`download_video(url)`
- Calls the `download_video` function, passing the URL as an argument.
- The `download_video` function is responsible for downloading the video from the specified URL.

#### Recording the Download
After the video is downloaded, the function retrieves the name of the current thread using `threading.current_thread().name`.
The `video_record` function is then called with the `url` and `thread_name` as arguments.
- This logs the URL and the name of the thread that handled the download, ensuring that each download is tracked.

In [58]:
# Extract the URLs from video_urls.txt
def extract_urls(youtube_videos):
    video_urls = []
    with open(youtube_videos, 'r', encoding='utf-8') as file:
        for url in file:
            url = url.strip()
            video_urls.append(url)
            # Regex to extract the name of the video
            pattern = r'[^/]+$'
            # Search the URL for the filename using the regex pattern
            videoename = re.search(pattern, url).group(0)
            videoenames.append(videoename)
    return video_urls

#### Function Definition
The function takes a single argument, `youtube_videos`, which is the path to a file containing YouTube video URLs.

#### Initializing the List for URLs
`video_urls = []`
- An empty list `video_urls` is initialized to store the extracted URLs.

#### Reading the File
`with open(youtube_videos, 'r', encoding='utf-8') as file:`
- The file specified by `youtube_videos` is opened in read mode with UTF-8 encoding.
- A for loop iterates over each line in the file, where each line is expected to contain a YouTube video URL.

#### Processing Each URL
`url = url.strip()`
- Each URL is stripped of leading and trailing whitespace using the `strip` method.
- The stripped URL is appended to the `video_urls` list.

#### Extracting the Video Name Using Regex
A regular expression pattern `r'[^/]+$'` is defined to match the part of the URL after the last forward slash, which is typically the video name or ID.
The `re.search` function searches the URL for a match to the regex pattern.
The `group(0)` method retrieves the matched string (the video name).
The extracted video name is appended to a global list `videoenames`.

#### Returning the List of URLs
The function returns the `video_urls` list, which contains all the extracted YouTube video URLs.

In [59]:
# Download video content
def download_video(url):
    try:
        yt = YouTube(url)
        stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
        filename = stream.download(output_path=download_directory)
        print(f'{url} downloaded.')
        extract_audio(filename, audio_directory)
    except Exception as e:
        print(f'Error downloading {url}: {e}')

#### Function Definition
The function takes a single argument, `url`, which is the URL of the YouTube video to be downloaded.

#### Try-Except Block
A `try` block is used to handle any exceptions that may occur during the video download and audio extraction process.

#### Initializing YouTube Object
`yt = YouTube(url)`
- A `YouTube` object is created using the `pytube` library, initialized with the given URL. This object provides access to various streams and metadata of the video.

#### Filtering and Selecting the Video Stream
`stream = yt.streams.filter(progressive=True, file_extension='mp4').first()`
- The `streams.filter` method is used to filter the available streams.
  - `progressive=True` ensures that the video contains both video and audio.
  - `file_extension='mp4'` filters the streams to those with an MP4 file extension.
- The `first()` method selects the first stream that matches the filter criteria.

#### Downloading the Video
`filename = stream.download(output_path=download_directory)`
- The selected stream is downloaded to the directory specified by `download_directory`.
- The `download` method returns the filename of the downloaded video.

#### Logging the Download
`print(f'{url} downloaded.')`
- A message is printed to the console to indicate that the video has been successfully downloaded.

#### Extracting Audio from the Video
`extract_audio(filename, audio_directory)`
- The `extract_audio` function is called with the filename of the downloaded video and the directory where the audio should be saved.
- This function extracts the audio from the downloaded video and saves it as a separate file.

#### Exception Handling
`except Exception as e: print(f'Error downloading {url}: {e}')`
- An `except` block catches any exceptions that occur during the download or audio extraction process.
- If an error occurs, an error message is printed to the console, including the URL and the exception message.

In [60]:
def serial_downloader(video_urls):
    start = time.perf_counter()
    for video in video_urls:
        download_video_and_log(video)
    end = time.perf_counter()
    print(f'Serial run took {round(end-start, 2)} second(s)')

#### Function Definition
The function takes a single argument, `video_urls`, which is a list of URLs of the videos to be downloaded.

#### Measuring Start Time
`start = time.perf_counter()`
- The `time.perf_counter()` function is used to record the start time of the operation. This high-resolution timer is suitable for measuring short durations.

#### Sequentially Downloading Videos
A for loop iterates over each URL in the `video_urls` list.
For each URL, the `download_video_and_log` function is called. This function handles the downloading of the video and logs the download details.

#### Measuring End Time
`end = time.perf_counter()`
- The `time.perf_counter()` function is used again to record the end time of the operation.

#### Calculating and Printing the Total Time
`print(f'Serial run took {round(end-start, 2)} second(s)')`
- The total time taken for the operation is calculated by subtracting the start time from the end time.
- The result is rounded to two decimal places and printed to the console.

In [61]:
def parallel_downloader(video_urls):
    start = time.perf_counter()
    processes = []
    for video in video_urls:
        p = multiprocessing.Process(target=download_video, args=(video,))
        p.start()
        processes.append(p)
    for p in processes:
        p.join()
    end = time.perf_counter()
    print(f'Parallel run took {round(end-start, 2)} second(s)')

#### Function Definition
The function takes a single argument, `video_urls`, which is a list of URLs of the videos to be downloaded.

#### Measuring Start Time
`start = time.perf_counter()`
- The `time.perf_counter()` function is used to record the start time of the operation. This high-resolution timer is suitable for measuring short durations.

#### Initializing the Process List
`processes = []`
- An empty list `processes` is initialized to store the process objects.

#### Creating and Starting Processes
A for loop iterates over each URL in the `video_urls` list.
For each URL, a new process is created using `multiprocessing.Process`.
- `target=download_video` specifies that the `download_video` function will be run in the new process.
- `args=(video,)` passes the URL as an argument to the `download_video` function.
The process is started using the `start()` method.
The process object is appended to the `processes` list.

#### Waiting for All Processes to Complete
A for loop iterates over each process in the `processes` list.
The `join()` method is called on each process, which blocks the main program until the process completes.
This ensures that the program waits for all video downloads to finish before proceeding.

#### Measuring End Time
`end = time.perf_counter()`
- The `time.perf_counter()` function is used again to record the end time of the operation.

#### Calculating and Printing the Total Time
`print(f'Parallel run took {round(end-start, 2)} second(s)')`
- The total time taken for the operation is calculated by subtracting the start time from the end time.
- The result is rounded to two decimal places and printed to the console.

In [62]:
def parallel_downloader1(video_urls):
    start = time.perf_counter()
    threads = []
    for video in video_urls:
        t = threading.Thread(target=download_video_and_log, args=(video,))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    end = time.perf_counter()
    print(f'Parallel run with threads took {round(end-start, 2)} second(s)')

#### Function Definition
The function takes a single argument, `video_urls`, which is a list of URLs of the videos to be downloaded.

#### Measuring Start Time
`start = time.perf_counter()`
- The `time.perf_counter()` function is used to record the start time of the operation. This high-resolution timer is suitable for measuring short durations.

#### Initializing the Thread List
`threads = []`
- An empty list `threads` is initialized to store the thread objects.

#### Creating and Starting Threads
A for loop iterates over each URL in the `video_urls` list.
For each URL, a new thread is created using `threading.Thread`.
- `target=download_video_and_log` specifies that the `download_video_and_log` function will be run in the new thread.
- `args=(video,)` passes the URL as an argument to the `download_video_and_log` function.
The thread is started using the `start()` method.
The thread object is appended to the `threads` list.

#### Waiting for All Threads to Complete
A for loop iterates over each thread in the `threads` list.
The `join()` method is called on each thread, which blocks the main program until the thread completes.
This ensures that the program waits for all video downloads to finish before proceeding.

#### Measuring End Time
`end = time.perf_counter()`
- The `time.perf_counter()` function is used again to record the end time of the operation.

#### Calculating and Printing the Total Time
`print(f'Parallel run with threads took {round(end-start, 2)} second(s)')`
- The total time taken for the operation is calculated by subtracting the start time from the end time.
- The result is rounded to two decimal places and printed to the console.

In [63]:
# Limit to 5 concurrent downloads
semaphore = threading.Semaphore(5)

In [64]:
def parallel_downloader2(video_urls):
    start = time.perf_counter()
    
    def download_with_semaphore(video):
        with semaphore:
            download_video_and_log(video)
    
    threads = []
    for video in video_urls:
        t = threading.Thread(target=download_with_semaphore, args=(video,))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    
    end = time.perf_counter()
    print(f'Parallel run with limited threads took {round(end-start, 2)} second(s)')

#### Function Definition
The function takes a single argument, `video_urls`, which is a list of URLs of the videos to be downloaded.

#### Measuring Start Time
`start = time.perf_counter()`
- The `time.perf_counter()` function is used to record the start time of the operation. This high-resolution timer is suitable for measuring short durations.

#### Nested Function with Semaphore
A nested function `download_with_semaphore` is defined within `parallel_downloader2`.
This function takes a single argument, `video`, which is a URL of a video to be downloaded.
A semaphore `semaphore` (assumed to be defined globally) is used to limit the number of concurrent threads.
The `with semaphore` statement ensures that the number of threads running `download_video_and_log` concurrently is limited by the semaphore.

#### Initializing the Thread List
`threads = []`
- An empty list `threads` is initialized to store the thread objects.

#### Creating and Starting Threads
A for loop iterates over each URL in the `video_urls` list.
For each URL, a new thread is created using `threading.Thread`.
- `target=download_with_semaphore` specifies that the `download_with_semaphore` function will be run in the new thread.
- `args=(video,)` passes the URL as an argument to the `download_with_semaphore` function.
The thread is started using the `start()` method.
The thread object is appended to the `threads` list.

#### Waiting for All Threads to Complete
A for loop iterates over each thread in the `threads` list.
The `join()` method is called on each thread, which blocks the main program until the thread completes.
This ensures that the program waits for all video downloads to finish before proceeding.

#### Measuring End Time
`end = time.perf_counter()`
- The `time.perf_counter()` function is used again to record the end time of the operation.

#### Calculating and Printing the Total Time
`print(f'Parallel run with limited threads took {round(end-start, 2)} second(s)')`
- The total time taken for the operation is calculated by subtracting the start time from the end time.
- The result is rounded to two decimal places and printed to the console.

In [65]:
# Extract audio from a videofile
def extract_audio(video_path, audio_directory):
    Path(audio_directory).mkdir(parents=True, exist_ok=True)
    
    try:
        video = VideoFileClip(video_path)
        video_path_obj = Path(video_path)
        audio_filename = video_path_obj.stem + '.wav'
        audio_path = Path(audio_directory) / audio_filename
        video.audio.write_audiofile(str(audio_path))
        print(f"Audio extracted and saved to {audio_path}")
        return audio_path
    except Exception as e:
        print(f"Failed to extract audio from {video_path}: {e}")
        return None
    finally:
        if video:
            video.close()

#### Function Definition
The function takes two arguments:
- `video_path`: The path to the video file from which audio needs to be extracted.
- `audio_directory`: The directory where the extracted audio file will be saved.

#### Creating the Audio Directory
`Path(audio_directory).mkdir(parents=True, exist_ok=True)`
- This line ensures that the `audio_directory` exists.
- Creates the directory and any necessary parent directories if they do not already exist.

#### Try-Except Block
A `try` block is used to handle any exceptions that may occur during the audio extraction process.

#### Loading the Video File
`video = VideoFileClip(video_path)`
- The `VideoFileClip` class from `moviepy.editor` is used to load the video file specified by `video_path`.

#### Constructing the Audio File Path
`video_path_obj = Path(video_path)`
- `video_path_obj` is a `Path` object created from `video_path`.
`audio_filename = video_path_obj.stem + '.wav'`
- `audio_filename` is constructed by taking the stem (the filename without the extension) of `video_path` and appending the `.wav` extension.
`audio_path = Path(audio_directory) / audio_filename`
- `audio_path` is the full path to the output audio file, constructed by combining `audio_directory` and `audio_filename`.

#### Extracting and Saving the Audio
`video.audio.write_audiofile(str(audio_path))`
- The `write_audiofile` method of the `audio` attribute of the `video` object is used to write the audio to the specified `audio_path`.
- A message is printed to the console indicating that the audio has been successfully extracted and saved.
- The function returns the path to the saved audio file.

#### Exception Handling
An `except` block catches any exceptions that occur during the audio extraction process.
If an error occurs, an error message is printed to the console, including the path to the video file and the exception message.
The function returns `None` to indicate that the audio extraction failed.

#### Ensuring the Video Object is Closed
The `finally` block ensures that the `video` object is closed, releasing any resources associated with it.
This is important for managing memory and preventing resource leaks.

In [66]:
# Transcribe audio from an audio file to text
def audio_to_text(audio_path, text_directory):
    Path(text_directory).mkdir(parents=True, exist_ok=True)
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(str(audio_path)) as source:
            audio = recognizer.record(source)
        text = recognizer.recognize_google(audio)
        print(f"Recognized text: {text}")
        # Perform the sentiment analysis on a video's content, extracting its polarity and sensitivity.
        sent_analysis(text)
        translator(text)
        emotion(text)
    except sr.UnknownValueError:
        print(f"Speech Recognition could not understand audio in {audio_path}")
        text = ""
    except sr.RequestError as e:
        print(f"Request error for: {e}")
        text = ""
    except Exception as e:
        print(f"Error processing audio file {audio_path}: {e}")
        text = ""
    
    audio_path_obj = Path(audio_path)
    text_filename = audio_path_obj.stem + '.txt'
    text_path = Path(text_directory) / text_filename
    
    try:
        with open(text_path, 'w', encoding='utf-8') as text_file:
            text_file.write(text)
        print(f"Text converted and saved to {text_path}")
    except Exception as e:
        print(f"Failed to save text file {text_path}: {e}")

#### Function Definition
The function takes two arguments:
- `audio_path`: The path to the audio file to be transcribed.
- `text_directory`: The directory where the transcribed text file will be saved.

#### Creating the Text Directory
`Path(text_directory).mkdir(parents=True, exist_ok=True)`
- This line ensures that the `text_directory` exists.
- Creates the directory and any necessary parent directories if they do not already exist.

#### Initializing the Recognizer
`recognizer = sr.Recognizer()`
- An instance of the `Recognizer` class from the `speech_recognition` library is created. This object will be used to recognize speech from the audio file.

#### Try-Except Block for Transcription
A `try` block is used to handle any exceptions that may occur during the transcription process.
The `with sr.AudioFile(str(audio_path)) as source` statement opens the audio file for reading.
The `recognizer.record(source)` method records the audio from the file.
The `recognizer.recognize_google(audio)` method transcribes the recorded audio to text using Google's speech recognition service.
If the transcription is successful, a log message is generated, and the recognized text is printed to the console.
The text is then analyzed for sentiment, translated, and checked for emotions using the respective functions: `sent_analysis`, `translator`, and `emotion`.

#### Handling Specific Exceptions
The `except sr.UnknownValueError` block handles cases where the speech recognition service could not understand the audio.
The `except sr.RequestError` block handles cases where there is a request error when calling the speech recognition service.
A general `except Exception` block catches any other exceptions that may occur.

#### Constructing the Text File Path
`audio_path_obj = Path(audio_path)`
- `audio_path_obj` is a `Path` object created from `audio_path`.
`text_filename = audio_path_obj.stem + '.txt'`
- `text_filename` is constructed by taking the stem (the filename without the extension) of `audio_path` and appending the `.txt` extension.
`text_path = Path(text_directory) / text_filename`
- `text_path` is the full path to the output text file, constructed by combining `text_directory` and `text_filename`.

#### Saving the Transcribed Text
A `try` block is used to handle any exceptions that may occur during the file writing process.
The `with open(text_path, 'w', encoding='utf-8') as text_file` statement opens the text file for writing with UTF-8 encoding.
The transcribed text is written to the file.
If the text is successfully saved, a log message is generated, and a message is printed to the console.
If an error occurs, an error message is logged and printed to the console.

In [67]:
# Perform sentiment analysis on a given text 
def sent_analysis(text):
    blob = TextBlob(text)
    try: 
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        print('Sentiment extracted from', text, 'resulted in', polarity, 'for polarity and', subjectivity, 'for subjectivity')
    except Exception as e:
        print(f"The text could not be extracted. An error occured: {e}")

#### Function Definition
The function takes a single argument, `text`, which is the text to be analyzed for sentiment.

#### Creating a TextBlob Object
`blob = TextBlob(text)`
- A `TextBlob` object is created using the `TextBlob` class from the `textblob` library, initialized with the given text. This object provides methods for natural language processing, including sentiment analysis.

#### Try-Except Block for Sentiment Analysis
A `try` block is used to handle any exceptions that may occur during sentiment analysis.
`blob.sentiment.polarity` extracts the polarity of the text. Polarity is a float within the range [-1.0, 1.0], where -1 indicates negative sentiment, 0 indicates neutral sentiment, and 1 indicates positive sentiment.
`blob.sentiment.subjectivity` extracts the subjectivity of the text. Subjectivity is a float within the range [0.0, 1.0], where 0 is very objective and 1 is very subjective.
The results of the sentiment analysis, including polarity and subjectivity, are printed to the console.
If an error occurs during sentiment analysis, an error message is printed to the console, indicating that the text could not be extracted and providing details about the exception.

In [68]:
# Translate a given text from English to Italian 
def translator(text):
    try:
        blob = TextBlob(text)
        translator = Translator()
        blob_translated = translator.translate(text, src='en', dest='it').text
        print(f'{text} was translated in Italian as follows: {blob_translated}')
    except Exception as e:
        print(f"The text could not be translated. An error occurred: {e}")
        

#### Function Definition
The function takes a single argument, `text`, which is the text to be translated.

#### Try-Except Block for Translation
A `try` block is used to handle any exceptions that may occur during the translation process.
`blob = TextBlob(text)` creates a `TextBlob` object from the `text`. This step is not necessary for translation but might be a placeholder for additional processing.
`translator = Translator()` creates an instance of the `Translator` class from the `googletrans` library.
`blob_translated = translator.translate(text, src='en', dest='it').text` translates the text from English (`src='en'`) to Italian (`dest='it'`) and extracts the translated text.
If the translation is successful, the original text and its translation are printed to the console.
If an error occurs during translation, an error message is printed to the console, indicating that the text could not be translated and providing details about the exception.

In [69]:
# Perform emotion analysis on a given text
def emotion(text):
    nlp = spacy.load('en_core_web_sm')
    nltk.download('punkt')
    doc = nlp(text)
    full_text = ' '.join([sent.text for sent in doc.sents])
    emotion = NRCLex(text)
    print("Detected Emotions and Frequencies:")
    print(emotion.affect_frequencies)

#### Function Definition
The function takes a single argument, `text`, which is the text to be analyzed for emotions.

#### Loading Spacy Model
`nlp = spacy.load('en_core_web_sm')`
- The `spacy.load('en_core_web_sm')` function loads the small English language model provided by Spacy, which is used for various natural language processing tasks.

#### Downloading NLTK Punkt
`nltk.download('punkt')`
- The `nltk.download('punkt')` function downloads the Punkt tokenizer models, which are used for sentence splitting.

#### Processing the Text with Spacy
`doc = nlp(text)`
- The `nlp(text)` function processes the input `text` using the Spacy model, resulting in a `doc` object that contains the parsed text.

#### Extracting Sentences
A list comprehension is used to extract the text of each sentence in the `doc` object.
The extracted sentences are joined into a single string, `full_text`.

#### Performing Emotion Analysis
An `NRCLex` object is created using the `NRCLex` class from the `nrclex` library, initialized with the input `text`.
This object is used to analyze the emotions present in the text.

#### Printing Detected Emotions and Frequencies
The detected emotions and their frequencies are printed to the console.
`emotion.affect_frequencies` is a dictionary containing the frequencies of different emotions detected in the text.

In [ ]:
if __name__ == '__main__':
    urls_YouTube = [
        'https://www.youtube.com/shorts/CS3kviWGkH0',
        'https://www.youtube.com/shorts/-WowH0liGfE',
        'https://www.youtube.com/shorts/QTyQtQmpZ0g',
        'https://www.youtube.com/shorts/KDqXneUN4CA',
        'https://www.youtube.com/shorts/AnOJRqLnfqQ',
        'https://www.youtube.com/shorts/LcQa94wX-O8',
        'https://www.youtube.com/shorts/z-F3cQ7YN28',
        'https://www.youtube.com/shorts/j_Pv6EHYo9k',
        'https://www.youtube.com/shorts/ZTdXLfHWDJE',
        'https://www.youtube.com/shorts/x0iC-8ZnNwM'
    ]
    
    youtube_videos = 'video_urls.txt'
    with open(youtube_videos, 'w', encoding='utf-8') as file:
        for text in urls_YouTube:
            file.write(text + '\n')
    video_urls = extract_urls(youtube_videos)
    print(videoenames)
    serial_downloader(video_urls)
    parallel_downloader(video_urls)
    parallel_downloader1(video_urls)
    parallel_downloader2(video_urls)

    # Transcribe audio to text after all downloads and audio extractions
    for audio_file in Path(audio_directory).glob('*.wav'):
        audio_to_text(audio_file, text_directory)
    

['CS3kviWGkH0', '-WowH0liGfE', 'QTyQtQmpZ0g', 'KDqXneUN4CA', 'AnOJRqLnfqQ', 'LcQa94wX-O8', 'z-F3cQ7YN28', 'j_Pv6EHYo9k', 'ZTdXLfHWDJE', 'x0iC-8ZnNwM']
https://www.youtube.com/shorts/CS3kviWGkH0 downloaded.
MoviePy - Writing audio in audios/My Talking Dog Uses Her Buttons to Talk About The Past   Talking Dog Buttons.wav


MoviePy - Done.
Audio extracted and saved to audios/My Talking Dog Uses Her Buttons to Talk About The Past   Talking Dog Buttons.wav
https://www.youtube.com/shorts/-WowH0liGfE downloaded.
MoviePy - Writing audio in audios/Your daily dose of funny cute dogs relaxmydog funny dogs funnydogs compilation.wav


MoviePy - Done.
Audio extracted and saved to audios/Your daily dose of funny cute dogs relaxmydog funny dogs funnydogs compilation.wav
Error downloading https://www.youtube.com/shorts/QTyQtQmpZ0g: QTyQtQmpZ0g is age restricted, and can't be accessed without logging in.
https://www.youtube.com/shorts/KDqXneUN4CA downloaded.
MoviePy - Writing audio in audios/Border Collie 🐶 The Most Intelligent Dog Breed In The World shorts bordercollie dog.wav


MoviePy - Done.
Audio extracted and saved to audios/Border Collie 🐶 The Most Intelligent Dog Breed In The World shorts bordercollie dog.wav
https://www.youtube.com/shorts/AnOJRqLnfqQ downloaded.
MoviePy - Writing audio in audios/Come Talk!  My talking dog uses buttons to communicate  WhatAboutBunny.wav


MoviePy - Done.
Audio extracted and saved to audios/Come Talk!  My talking dog uses buttons to communicate  WhatAboutBunny.wav
https://www.youtube.com/shorts/LcQa94wX-O8 downloaded.
MoviePy - Writing audio in audios/sadpitbullturnedhappy                                CTTO.wav


...

Error downloading https://www.youtube.com/shorts/ZTdXLfHWDJE: ZTdXLfHWDJE is age restricted, and can't be accessed without logging in.



chunk:  79%|████████████████████████████▍       | 771/978 [00:01<00:00, 888.31it/s, now=None]
                                                                                             
                                                                                          
chunk:  47%|████████████████▌                  | 627/1325 [00:00<00:01, 537.47it/s, now=None]
                                                                                          
chunk:  51%|█████████████████▉                 | 678/1325 [00:00<00:01, 537.47it/s, now=None]

https://www.youtube.com/shorts/x0iC-8ZnNwM downloaded.
MoviePy - Done.
Audio extracted and saved to audios/TALKING DOG has BIG FEELINGS! shorts dogshorts talkingdog smartdog.wav
MoviePy - Writing audio in audios/Laugh Out Loud at These Funny Dog Moments! 😄.wav


chunk:   0%|                                               | 0/178 [00:00<?, ?it/s, now=None]
                                                                                             
                                                                                          
chunk:  75%|████████████████████████▉        | 1000/1325 [00:01<00:00, 1006.18it/s, now=None]
                                                                                             

MoviePy - Done.
Audio extracted and saved to audios/Laugh Out Loud at These Funny Dog Moments! 😄.wav
MoviePy - Done.
Audio extracted and saved to audios/Morning routine 🥹.wav
Parallel run with limited threads took 11.93 second(s)


#### Writing URLs to a File
The URLs are written to a file named `video_urls.txt`.
This file is opened in write mode (`'w'`) with UTF-8 encoding.
Each URL from the `urls_YouTube` list is written to the file on a new line.

#### Extracting URLs from the File
The `extract_urls` function is called with the filename `video_urls.txt` as an argument. This function reads the URLs from the file and returns a list of URLs.
The list of extracted video names (`videoenames`) is printed to the console.

#### Downloading Videos
The videos are downloaded using different strategies:
- `serial_downloader`: Downloads videos sequentially.
- `parallel_downloader`: Downloads videos in parallel using multiple processes.
- `parallel_downloader1`: Downloads videos in parallel using multiple threads.
- `parallel_downloader2`: Downloads videos in parallel using multiple threads with a limit on the number of concurrent threads (using a semaphore).

#### Transcribing Audio to Text
After all downloads and audio extractions are complete, the script transcribes the audio files to text.
The `audio_to_text` function is called for each `.wav` audio file in the `audio_directory`.